In [1]:
import os
import requests
import json
import csv
import pandas as pd
import sys
#This may need to be pathlib under python 3
import pathlib2  

YOUR_API_KEY = '[PUT YOUR API KEY HERE]'

API_KEY = os.getenv('API_KEY', YOUR_API_KEY)

# progress bar

def progress(count, total, status=''):
    bar_len = 35
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s %s %s-%s\r' % (bar, percents, '%', status, count, (count + 999)))

    sys.stdout.flush() 

In [ ]:
doc_count_url = 'https://api.data.gov:443/regulations/v3/documents.json?api_key=%s&countsOnly=1&dct=PS&dktid=DOI-2017-0002' % API_KEY
r = requests.get(doc_count_url)
if r.status_code == 200:
    result = r.json()
    records = result['totalNumRecords']
    print 'Records available: {}'.format(records)

In [ ]:
# get the previous comments
offset = 0
comments = 'dataset/comments.csv'

#if comments.csv exists, count rows
from pathlib2 import Path
my_file = Path(comments)
if my_file.is_file():
    df = pd.read_csv(comments)
    offset = df.shape[0]
    
print 'Starting with comment: {}'.format(offset)

In [ ]:
documents_url = 'https://api.data.gov:443/regulations/v3/documents.json?api_key=%s&dct=PS&dktid=DOI-2017-0002&rpp=1000&po=%s&sb=docId&so=ASC'
documents = list()

#add a check to avoid API overload. Don't download more than 1 million (1000 API calls) in an hour.
if records - offset > 1000000:
    records = 900000
    print '*****HEADS UP!*****'
    print 'Only downloading a partial set of comments.'
    print 'This will stop after 990,000 to avoid the API limit.'
    print 'Run this again in an hour to get the rest.'
    print '*****HEADS UP!*****'

while offset < records :
    progress(offset, records, status='Downloading comments')
    r = requests.get(documents_url % (API_KEY, offset))
    if r.status_code == 200:
        result = r.json()
        documents = documents + result['documents']
        offset += 1000
    else:
        raise Exception("non 200 response code").with_traceback(tracebackobj)

progress(records, records, status='Downloading comments')
print '\n\nDone!'

In [ ]:
field_names = ['documentId', 'postedDate', 'attachmentCount', 'commentText']

# write headers if this is a new file
if not my_file.is_file():
    with open(comments, 'w') as f:
        writer = csv.DictWriter(f, field_names, extrasaction='ignore')
        writer.writeheader()

with open(comments, 'a') as f:
    writer = csv.DictWriter(f, field_names, extrasaction='ignore')
    writer.writerows(documents)

In [ ]:
#reality check
df = pd.read_csv(comments)
print 'Rows in comments.csv: {}'.format(df.shape[0])